#Invoke AI Notebook

Tested: Generating images with the SDXL base model.

Not tested: Using the refiner.

Broken: Adding custom checkpoints.

In [ ]:
#@markdown # Downloading SDLX Models and Installing InvokeAI

# Models folder
autoimportPath = '/content/invokeai/autoimport/main'

#@markdown Use Google Drive to store models (uses about 13 GB). Uncheck this if you don't have enough space in your Drive.
useGoogleDrive = True #@param {type:"boolean"}

googleDriveModelsFolder = '/stablemodels' #@param {type:"string"}

#@markdown This step usually takes about 5 minutes.

from google.colab import drive
if useGoogleDrive:
  drive.mount('/content/drive')
  if not googleDriveModelsFolder.startswith('/'):
    googleDriveModelsFolder = '/' + googleDriveModelsFolder
  modelsPath = "/content/drive/MyDrive"+googleDriveModelsFolder
  if not modelsPath.endswith("/"):
   modelsPath = modelsPath + "/"

!pip install 'InvokeAI[xformers]' --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu117



In [ ]:
#@markdown # Downloading Models

!mkdir /content/invokeai
!mkdir /content/invokeai/configs

#@markdown Download only the default model in initial configuration.
#@markdown Checking this prevents running out of space in Colab.

defaultOnly = True #@param {type:"boolean"}
skipWeights = True #@param {type:"boolean"}

#@markdown This step usually takes about 2 minutes with only the default model and no weights.

#@markdown You can ignore "File exists" warnings in the output.

import os
import subprocess
env = os.environ.copy()

cmd = 'invokeai-configure --root_dir /content/invokeai --yes'

if defaultOnly:
  cmd += ' --default_only'

if skipWeights:
  cmd += ' --skip-sd-weights'

subprocess.run(cmd, shell=True, env=env)




In [ ]:

#@markdown Adding the SDXL Base Model

#@markdown Installing SDXL base took about 20 minutes initially, but it's finished instantly
#@markdown in subsequent runs if Google Drive is enabled. You can run this in a runtime
#@markdown without a GPU to save the model to Google Drive without spending GPU time.

# If we don't have SDLX in Drive, create the folder

import os.path
from os import path

# Install the SDXL base model
def installSdxl():
  installCmd = 'invokeai-model-install --add "stabilityai/stable-diffusion-xl-base-1.0" --root_dir /content/invokeai'
  subprocess.run(installCmd, shell=True, env=env)

if useGoogleDrive:
  alreadyInstalled = True

  driveSdxlBasePath = modelsPath + "sdxl/main"
  if not path.exists(driveSdxlBasePath):
    os.makedirs(driveSdxlBasePath, exist_ok=True)
    alreadyInstalled = False

  localSdxlBaseFolder = "/content/invokeai/models/sdxl/"
  localSdxlBasePath = localSdxlBaseFolder + "main"

  subprocess.run('rm -rf ' + localSdxlBaseFolder, shell=True, env=env)
  subprocess.run('rmdir ' + localSdxlBaseFolder, shell=True, env=env)

  if not path.exists(localSdxlBaseFolder):
    os.makedirs(localSdxlBaseFolder, exist_ok=True)

  subprocess.run('ln -s '+driveSdxlBasePath+' '+localSdxlBaseFolder, shell=True, env=env)

  if not alreadyInstalled:
    installSdxl()
  else:
    with open("/content/invokeai/configs/models.yaml", "a") as file:
      lines = [
        "sdxl/main/stable-diffusion-xl-base-1-0:\n",
        "  path: sdxl/main/stable-diffusion-xl-base-1-0\n",
        "  description: Stable Diffusion XL base model (12 GB)\n",
        "  variant: normal\n",
        "  format: diffusers\n"
      ]
      file.writelines(lines)
else:
  installSdxl()



In [ ]:
#@markdown Adding the refiner and vae. This one took about 14 minutes.
#@markdown Skip this step if you don't need these models.
!invokeai-model-install --add "stabilityai/stable-diffusion-xl-refiner-1.0" --root_dir /content/invokeai
!invokeai-model-install --add "madebyollin/sdxl-vae-fp16-fix" --root_dir /content/invokeai

In [ ]:
#@markdown # Starting the Web UI

%cd /content/invokeai/
!npm install -g localtunnel

#@markdown Copy the IP address shown in the output above the line
#@markdown "your url is: https://some-random-words.loca.lt"
!wget -q -O - ipv4.icanhazip.com

#@markdown Wait for the line that says "Uvicorn running on http://127.0.0.1:9090 (Press CTRL+C to quit)"

#@markdown Click the localtunnel url and paste the IP you copied earlier to the "Endpoint IP" text field
!lt --port 9090 --local_https False & invokeai-web  --root /content/invokeai/

#@markdown If the UI shows a red dot that says 'disconnected' when hovered in the upper
#@markdown right corner and the Invoke button is disabled, change 'https' to 'http'
#@markdown in the browser's address bar and press enter.
#@markdown When the page reloads, the UI should work properly.
